In [ ]:
from engine import TradingEngine
import pandas as pd

bot = TradingEngine()

# Determine testing ranges
testing_ranges = [5, 10, 20, 50, 100, 200]
opt_results = []

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)

for v in testing_ranges:
    # Do not manipulate original DataFrame
    df_temp = df_base.copy()
    
    # Run backtest with current window
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    
    # Store result
    opt_results.append({
        'Window': v,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance']
    })

# Convert to DataFrame for comparison
df_results = pd.DataFrame(opt_results)
print(df_results.sort_values(by='Return %', ascending=False))

In [ ]:
import matplotlib.pyplot as plt

# 1. Determine range (5, 10, 15... 300)
windows = range(5, 301, 5)
mReturns = []

df_base = bot.fetch_data(timeframe='1h', limit=2000) # More data = better statistics

for v in windows:
    df_temp = df_base.copy()
    df_temp = bot.add_indicators(df_temp, sma_window=v)
    df_temp = bot.get_signals(df_temp, sma_window=v)
    res = bot.run_backtest(df_temp)
    mReturns.append(res['total_return_pct'])

# 2. Plot optimisation curve
plt.figure(figsize=(12, 6))
plt.plot(windows, mReturns, marker='o', linestyle='-', color='b', markersize=4)
plt.axhline(0, color='red', linestyle='--') # Equilibrium line (0% profit)

plt.title('Parameter optimisation: SMA window vs Profitability')
plt.xlabel('SMA window size')
plt.ylabel('Total Return (%)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from engine import TradingEngine
import pandas as pd

symbol = 'BTC/USDT'  # Bitcoin
# symbol = 'LTC/USDT'  # Litecoin
# symbol = 'ETH/USDT'   # Ethereum
# symbol = 'ADA/USDT' # Cardano
# symbol = 'SOL/USDT' # Solana


bot = TradingEngine(symbol=symbol)  

# Download data once
df_base = bot.fetch_data(timeframe='1h', limit=1000)
df_base = bot.add_indicators(df_base, sma_window=10, ema_window=10, rsi_window=14, bb_window=20)

# Define parameter combinations to test
combinations = [
    {'sma_window':10, 'ema_window':None, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':False, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':False},
    {'sma_window':10, 'ema_window':10, 'use_macd':True, 'use_bbands':True},
]

results = []

# Iterate over combinations
for params in combinations:
    df_temp = df_base.copy()
    df_temp = bot.get_signals(
        df_temp,
        sma_window=params['sma_window'],
        ema_window=params['ema_window'],
        use_macd=params['use_macd'],
        use_bbands=params['use_bbands'],
        rsi_overbought=60,
        rsi_oversold=50
    )
    # Run backtest
    res = bot.run_backtest(df_temp)
    results.append({
        **params,
        'Return %': res['total_return_pct'],
        'Final $': res['final_balance'],
        'Max Drawdown %': res['df']['drawdown'].min()*100
    })

# Turn into DataFrame for comparison
df_results = pd.DataFrame(results)
df_results.sort_values(by='Return %', ascending=False)


In [ ]:
import mplfinance as mpf
import numpy as np
import pandas as pd
from engine import TradingEngine

symbol = 'BTC/USDT'  # Bitcoin
# symbol = 'LTC/USDT'  # Litecoin
# symbol = 'ETH/USDT'   # Ethereum
# symbol = 'ADA/USDT' # Cardano
# symbol = 'SOL/USDT' # Solana



# Fetch original data
bot = TradingEngine(symbol=symbol)
df = bot.fetch_data(timeframe='1h', limit=1000)
# Compute indicators & signals
df = bot.add_indicators(df, sma_window=20, ema_window=20, rsi_window=14, bb_window=20)
df = bot.get_signals(df, sma_window=20, rsi_overbought=60, rsi_oversold=50, use_macd=True, use_bbands=True)

# Copy and prepare data
df_plot = df.copy()
df_plot.set_index('timestamp', inplace=True)

# Buy/sell signs
df_plot['Signal_Diff'] = df_plot['Signal'].diff()

df_plot['buy_price'] = np.where(df_plot['Signal_Diff'] > 0, df_plot['close'] * 0.98, np.nan)
df_plot['sell_price'] = np.where(df_plot['Signal_Diff'] < 0, df_plot['close'] * 1.02, np.nan)

# Focus on recent data
df_plot_recent = df_plot.tail(300).copy()
df_plot_recent.dropna(subset=['MACD', 'MACD_Signal', 'RSI'], inplace=True)

# Addplots for each indicator

apds = [
    mpf.make_addplot(df_plot_recent['SMA_20'], color='orange', width=1.5, panel=0),
    mpf.make_addplot(df_plot_recent['EMA_20'], color='blue', width=1.5, panel=0),
    mpf.make_addplot(df_plot_recent['BB_High'], color='red', width=0.8, panel=0, alpha=0.5),
    mpf.make_addplot(df_plot_recent['BB_Low'], color='green', width=0.8, panel=0, alpha=0.5),
    mpf.make_addplot(df_plot_recent['MACD'], color='purple', panel=1, ylabel='MACD'),
    mpf.make_addplot(df_plot_recent['MACD_Signal'], color='black', panel=1),
    mpf.make_addplot(df_plot_recent['RSI'], color='brown', panel=2, ylabel='RSI'),
]

if not df_plot_recent['buy_price'].dropna().empty:
    apds.append(mpf.make_addplot(df_plot_recent['buy_price'], type='scatter', markersize=100, marker='^', color='green', panel=0))

if not df_plot_recent['sell_price'].dropna().empty:
    apds.append(mpf.make_addplot(df_plot_recent['sell_price'], type='scatter', markersize=100, marker='v', color='red', panel=0))


# Plot
mpf.plot(df_plot_recent,
         type='candle',
         style='charles',
         title=f"\n{bot.symbol} - Analysis",
         ylabel='Price (USDT)',
         volume=True,
         addplot=apds,
         figsize=(16,10),
         panel_ratios=(3,1,1),
         tight_layout=True)

In [ ]:
import time
import datetime
from engine import TradingEngine

# 1. Configuration
SYMBOL = 'BTC/USDT'
TIMEFRAME = '1m'  # Using 1m for faster testing in the notebook
INTERVAL = 30     # Seconds between updates
SMA_WINDOW = 20

# 2. Initialization
bot = TradingEngine(symbol=SYMBOL)
is_position_open = False
entry_price = 0.0
cumulative_pnl = 0.0
last_signal = 0

print(f"--- Starting Live Simulation in Notebook: {SYMBOL} ---")
print("Click the 'Stop' button to interrupt the loop.\n")

try:
    # Use a limited range for testing, or 'while True' for continuous monitoring
    for _ in range(50): 
        # A. Data Acquisition
        raw_df = bot.fetch_data(timeframe=TIMEFRAME, limit=100)
        df = bot.add_indicators(raw_df, sma_window=SMA_WINDOW)
        df = bot.get_signals(df, sma_window=SMA_WINDOW)
        
        last_row = df.iloc[-1]
        current_price = last_row['close']
        current_signal = last_row['Signal']
        timestamp = datetime.datetime.now().strftime('%H:%M:%S')

        # B. Simulation Logic
        if current_signal != last_signal:
            if current_signal == 1 and not is_position_open:
                entry_price = current_price
                is_position_open = True
                print(f"\n[{timestamp}] 🟢 SIGNAL: BUY at {entry_price}")
            
            elif current_signal == -1 and is_position_open:
                trade_pnl = (current_price - entry_price) / entry_price
                cumulative_pnl += trade_pnl
                is_position_open = False
                print(f"\n[{timestamp}] 🔴 SIGNAL: SELL at {current_price}")
                print(f"      Trade Return: {trade_pnl*100:.2f}% | Total PnL: {cumulative_pnl*100:.2f}%")
            
            last_signal = current_signal

        # C. Live Status Update
        unrealized_pnl = 0.0
        if is_position_open:
            unrealized_pnl = (current_price - entry_price) / entry_price
        
        # Display live price and PnL
        print(f"\r[{timestamp}] Price: {current_price:.2f} | Live PnL: {unrealized_pnl*100:.2f}% | Sig: {current_signal}", end="", flush=True)
        
        # Log
        bot.log_status(current_price, current_signal, unrealized_pnl, cumulative_pnl, is_position_open)
        
        time.sleep(INTERVAL)

except KeyboardInterrupt:
    print("\n\n--- Monitor stopped by user ---")
except Exception as e:
    print(f"\n\n[ERROR]: {e}")